# **Building and Evaluating an Advanced Retrieval-Augmented Generation (RAG) Pipeline**

# **Introduction**

This notebook demonstrates how to build and evaluate an advanced Retrieval-Augmented Generation (RAG) pipeline using LlamaIndex and TruLens. The pipeline includes multiple retrieval methods, such as Sentence Window retrieval and Auto-merging retrieval, with a focus on evaluating the quality of generated responses using various feedback metrics.

**Setup**

**Install Required Libraries**

Install the necessary Python libraries required for building and evaluating the RAG pipeline.

In [1]:
!pip install openai
!pip install llama_index
%pip install llama-index-embeddings-huggingface
%pip install llama-index-embeddings-instructor
!pip install trulens-eval

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.9/362.9 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 68.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 52.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.4/169.4 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 42.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.5 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully 

# **Basic RAG pipeline**

**API Key Setup**

Securely set up the OpenAI API key to interact with the GPT models.*

In [1]:
import openai
from getpass import getpass

# Prompt for the API key securely
openai.api_key = getpass("Enter your OpenAI API key:")


Enter your OpenAI API key:··········


**Document Loading and Preprocessing**

Load the documents to be used in the retrieval process. we are loading and merging multiple documents into a single document to improve text blending accuracy.

In [3]:
from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader(
    input_files=["./eBook-How-to-Build-a-Career-in-AI.pdf"]
).load_data()

In [4]:
print(type(documents), "\n")
print(len(documents), "\n")
print(type(documents[0]))
print(documents[0])



<class 'list'> 

41 

<class 'llama_index.core.schema.Document'>
Doc ID: 22d4e201-a94a-4da3-be29-48ff9ccdc783
Text: PAGE 1Founder, DeepLearning.AICollected Insights from Andrew Ng
How to  Build Your Career in AIA Simple Guide


In [5]:
# merging the 41 documentsinto a single document because it helps with overal text blendingv accuracy
# when using more advanced retrival methods

from llama_index.core import Document

document = Document(text="\n\n".join([doc.text for doc in documents]))

**Indexing the Documents**

Index the documents using a VectorStoreIndex and set up the LLM service context.

In [6]:
# Index these documents
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import VectorStoreIndex
from llama_index.core import ServiceContext
from llama_index.llms.openai import OpenAI

# we define a service context object thatll have both our LLm as well as our embedding model

llm = OpenAI(model="gpt-4o-mini", temperature=0.1)
service_context = ServiceContext.from_defaults(

    llm=llm, embed_model="local:BAAI/bge-small-en-v1.5"
)

index = VectorStoreIndex.from_documents([document],
                                        service_context=service_context)

<ipython-input-6-ecbb934a5f50>:10: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [7]:
#Query the indexed documents using the query engine.

query_engine = index.as_query_engine()

In [8]:
response = query_engine.query(
    "What are steps to take when finding projects to build your experience?"
)
print(str(response))

To find projects that can help build your experience, consider the following steps:

1. **Join Existing Projects**: Look for opportunities to collaborate on projects that others are already working on. This can provide valuable experience and insights.

2. **Engage in Continuous Learning**: Spend time reading, taking courses, and conversing with domain experts to generate new ideas and enhance your knowledge.

3. **Focus on Application Areas**: Identify specific application areas where machine learning can be applied, especially those that are less explored. This can lead to unique and creative projects.

4. **Develop a Side Hustle**: Consider starting a side project that aligns with your interests. This can be a fun way to experiment and may lead to significant opportunities.

5. **Evaluate Technical Growth**: Choose projects that challenge your skills but are achievable. This balance will help you grow technically.

6. **Collaborate with Good Teammates**: Work with individuals who ca

# **Evaluation Setup Using TruLens**


**Define Evaluation Questions**
Load the evaluation questions from a file.

In [9]:
#Load the evaluation questions from the txtfile
eval_questions = []
with open('eval_questions.txt', 'r') as file:
    for line in file:
        # Remove newline character and convert to integer
        item = line.strip()
        print(item)
        eval_questions.append(item)

What are the keys to building a career in AI?
How can teamwork contribute to success in AI?
What is the importance of networking in AI?
What are some good habits to develop for a successful career?
How can altruism be beneficial in building a career?
What is imposter syndrome and how does it relate to AI?
Who are some accomplished individuals who have experienced imposter syndrome?
What is the first step to becoming good at AI?
What are some common challenges in AI?
Is it normal to find parts of AI challenging?


In [10]:
from trulens_eval import (
    Feedback,
    TruLlama
)

from trulens_eval.feedback import GroundTruthAgreement

/usr/local/lib/python3.10/dist-packages/_distutils_hack/__init__.py:32: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
[nltk_data] Downloading package stopwords to
[nltk_data]     /usr/local/lib/python3.10/dist-
[nltk_data]     packages/llama_index/legacy/_static/nltk_cache...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to
[nltk_data]     /usr/local/lib/python3.10/dist-
[nltk_data]     packages/llama_index/legacy/_static/nltk_cache...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Initialize TruLens and set up feedback functions for evaluating the quality of generated responses.

Answer Relevance focuses on the direct relationship between the query and the response.

Context Relevance ensures that the response is appropriate within the broader context.

Groundedness checks the factual accuracy of the response relative to the source material.

Together, these feedback functions provide a comprehensive evaluation framework for tghe LLM, ensuring that it generates responses that are relevant, contextually appropriate, and factually accurate.

In [37]:
from trulens_eval import Tru
from trulens_eval.feedback.provider.openai import OpenAI as OpenAIProvider
from trulens_eval import Feedback, Select
import numpy as np
import nest_asyncio

nest_asyncio.apply()


# Securely set the API key using getpass
openai.api_key = getpass("Enter your OpenAI API key:")

# Initialize OpenAI provider for TruLens
provider = OpenAIProvider(api_key=openai.api_key)


# QA Relevance Feedback
f_qa_relevance = Feedback(
    provider.relevance_with_cot_reasons,
    name="Answer Relevance"
).on_input_output()

# Context Relevance Feedback
f_qs_relevance = Feedback(
    provider.relevance_with_cot_reasons,
    name="Context Relevance"
).on_input().on(Select.RecordCalls.retrieve.rets.collect()).aggregate(np.mean)

# Groundedness Feedback (Updated for v0.29.0)
f_groundedness = Feedback(
    provider.groundedness_measure_with_cot_reasons,
    name="Groundedness"
).on(Select.RecordCalls.retrieve.rets.collect()).on_output()

# Combine feedbacks into a list
feedbacks = [f_qa_relevance, f_qs_relevance, f_groundedness]


Enter your OpenAI API key:··········
✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input response will be set to __record__.app.retrieve.rets.collect() .
✅ In Groundedness, input source will be set to __record__.app.retrieve.rets.collect() .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .


In [38]:
# Initialize TruLens and reset the database
tru = Tru()
tru.reset_database()


In [39]:
# Function to create a TruLlama recorder
def get_trulens_recorder(query_engine, feedbacks, app_id):
    tru_recorder = TruLlama(
        query_engine,
        app_id=app_id,
        feedbacks=feedbacks
    )
    return tru_recorder

# Function to get a prebuilt TruLlama recorder (if needed)
def get_prebuilt_trulens_recorder(query_engine, app_id):
    tru_recorder = TruLlama(
        query_engine,
        app_id=app_id,
        feedbacks=feedbacks
    )
    return tru_recorder


**Recording and Evaluating Queries**

Use TruLens to record and evaluate the queries.

In [71]:
# Get the TruLlama recorder
tru_recorder = get_trulens_recorder(query_engine, feedbacks, app_id="Direct Query Engine")

# Use the TruLlama recorder
with tru_recorder as recording:
    for question in eval_questions:
        response = query_engine.query(question)

# Use the Tru object to retrieve records and feedback
tru = Tru()


In [41]:
# Retrieve records and their associated feedback
records, feedback = tru.get_records_and_feedback(app_ids=["Direct Query Engine"])


In [42]:
# Display the first few records
records.head()

,app_id,app_json,type,record_id,input,output,tags,record_json,cost_json,perf_json,ts,Answer Relevance,Groundedness,Context Relevance,Answer Relevance_calls,Groundedness_calls,Context Relevance_calls,latency,total_tokens,total_cost
0,Direct Query Engine,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.core.query_en...,record_hash_ebce5d49e29219fdb523d831c63e5354,"""What are the keys to building a career in AI?""","""The keys to building a career in AI include l...",-,"{""record_id"": ""record_hash_ebce5d49e29219fdb52...","{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2024-08-21T14:06:06.625825"", ""...",2024-08-21T14:06:09.429273,0.9,0.866667,0.9,[{'args': {'prompt': 'What are the keys to bui...,[{'args': {'source': [[{'node': {'id_': 'cdebf...,[{'args': {'prompt': 'What are the keys to bui...,2,0,0.0
1,Direct Query Engine,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.core.query_en...,record_hash_65ee3e6a05bce392cce492f7043c0713,"""How can teamwork contribute to success in AI?""","""Teamwork can significantly enhance success in...",-,"{""record_id"": ""record_hash_65ee3e6a05bce392cce...","{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2024-08-21T14:06:09.953843"", ""...",2024-08-21T14:06:13.462841,0.9,0.850000,0.8,[{'args': {'prompt': 'How can teamwork contrib...,[{'args': {'source': [[{'node': {'id_': 'b1b6c...,[{'args': {'prompt': 'How can teamwork contrib...,3,0,0.0
2,Direct Query Engine,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.core.query_en...,record_hash_2d832f4b9e43306b9ab62140e5b50ac7,"""What is the importance of networking in AI?""","""Networking is crucial in AI as it helps build...",-,"{""record_id"": ""record_hash_2d832f4b9e43306b9ab...","{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2024-08-21T14:06:13.963909"", ""...",2024-08-21T14:06:17.033638,0.8,0.833333,0.8,[{'args': {'prompt': 'What is the importance o...,[{'args': {'source': [[{'node': {'id_': 'b1b6c...,[{'args': {'prompt': 'What is the importance o...,3,0,0.0
3,Direct Query Engine,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.core.query_en...,record_hash_8ade281216cf5419f268bf6f5af68ce8,"""What are some good habits to develop for a su...","""Developing good habits in various areas can s...",-,"{""record_id"": ""record_hash_8ade281216cf5419f26...","{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2024-08-21T14:06:17.770827"", ""...",2024-08-21T14:06:21.043929,0.9,0.733333,0.9,[{'args': {'prompt': 'What are some good habit...,[{'args': {'source': [[{'node': {'id_': 'b1b6c...,[{'args': {'prompt': 'What are some good habit...,3,0,0.0
4,Direct Query Engine,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.core.query_en...,record_hash_3574b04b8232c19cf3b371741b96f7af,"""How can altruism be beneficial in building a ...","""Altruism can be beneficial in building a care...",-,"{""record_id"": ""record_hash_3574b04b8232c19cf3b...","{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2024-08-21T14:06:21.574677"", ""...",2024-08-21T14:06:26.794693,0.9,0.800000,0.8,[{'args': {'prompt': 'How can altruism be bene...,[{'args': {'source': [[{'node': {'id_': 'b1b6c...,[{'args': {'prompt': 'How can altruism be bene...,5,0,0.0


In [43]:
feedback

['Answer Relevance', 'Context Relevance', 'Groundedness']

In [44]:
# launches on http://localhost:8501/
tru.run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.
Dashboard already running at path:   Submit this IP Address: 35.229.251.83



<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

# **Advanced RAG pipeline**

# **1. Sentence Window retrieval**

The "Sentence-Window Retrieval Pipeline" is a method designed to enhance the retrieval process by focusing on smaller, contextually relevant chunks of information.

**Input:** The pipeline starts with documents that are segmented into smaller chunks.

**Embedding:** These chunks are converted into embeddings, which are then used to retrieve the most relevant chunks based on a query.

**Contextual Retrieval:** The context around the retrieved chunks is added to ensure the completeness of the information.

**Synthesis:** The selected chunks and their context are synthesized by an LLM to generate the final response.

In [45]:
from llama_index.core.node_parser import SentenceWindowNodeParser
from llama_index.core.indices.postprocessor import MetadataReplacementPostProcessor
from llama_index.core.indices.postprocessor import SentenceTransformerRerank
from llama_index.core import load_index_from_storage
import os
from llama_index.core import StorageContext


def build_sentence_window_index(
    document, llm, embed_model="local:BAAI/bge-small-en-v1.5", save_dir="sentence_index"
):
    # create the sentence window node parser w/ default settings
    node_parser = SentenceWindowNodeParser.from_defaults(
        window_size=3,
        window_metadata_key="window",
        original_text_metadata_key="original_text",
    )
    sentence_context = ServiceContext.from_defaults(
        llm=llm,
        embed_model=embed_model,
        node_parser=node_parser,
    )
    if not os.path.exists(save_dir):
        sentence_index = VectorStoreIndex.from_documents(
            [document], service_context=sentence_context
        )
        sentence_index.storage_context.persist(persist_dir=save_dir)
    else:
        sentence_index = load_index_from_storage(
            StorageContext.from_defaults(persist_dir=save_dir),
            service_context=sentence_context,
        )

    return sentence_index

In [46]:
sentence_index = build_sentence_window_index(
    document,
    llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    save_dir="sentence_index"
)

<ipython-input-45-bb5d973941ac>:18: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  sentence_context = ServiceContext.from_defaults(


In [47]:
def get_sentence_window_query_engine(
    sentence_index,
    similarity_top_k=6,
    rerank_top_n=2,
):
    # define postprocessors
    postproc = MetadataReplacementPostProcessor(target_metadata_key="window")
    rerank = SentenceTransformerRerank(
        top_n=rerank_top_n, model="BAAI/bge-reranker-base"
    )

    sentence_window_engine = sentence_index.as_query_engine(
        similarity_top_k=similarity_top_k, node_postprocessors=[postproc, rerank]
    )
    return sentence_window_engine


In [48]:
sentence_window_engine = get_sentence_window_query_engine(sentence_index)

In [49]:
window_response = sentence_window_engine.query(
    "how do I get started on a personal project in AI?"
)
print(str(window_response))

To get started on a personal project in AI, begin by identifying a project that aligns with your interests and career goals. It's important to choose a project that is responsible, ethical, and beneficial to people. Start small, as this will allow you to gradually build your skills and confidence. Consider the scope and complexity of the project, ensuring it provides an opportunity for growth. As you progress, document your work to create a portfolio that showcases your skill development. This approach will not only enhance your expertise but also prepare you for future opportunities in the field.


In [50]:
tru.reset_database()

tru_recorder_sentence_window = get_prebuilt_trulens_recorder(
    sentence_window_engine,
    app_id = "Sentence Window Query Engine"
)

In [51]:
for question in eval_questions:
    with tru_recorder_sentence_window as recording:
        response = sentence_window_engine.query(question)
        print(question)
        print(str(response))

What are the keys to building a career in AI?
The keys to building a career in AI include learning foundational technical skills, working on projects that demonstrate skill progression, and actively seeking job opportunities. Additionally, being part of a supportive community can enhance your journey and provide valuable connections and resources.
How can teamwork contribute to success in AI?
Teamwork can significantly contribute to success in AI by fostering collaboration and leveraging diverse skills and perspectives. When individuals work together, they can share insights, challenge each other's ideas, and identify potential flaws in reasoning, which enhances the overall quality of the project. Additionally, having colleagues who are dedicated to continuous learning and building beneficial AI solutions can create a motivating environment that encourages everyone to strive for excellence. Effective teamwork also allows for the pooling of technical expertise, enabling the team to make

In [52]:
tru.get_leaderboard(app_ids=[])

,Answer Relevance,Context Relevance,Groundedness,latency,total_cost
app_id,,,,,
Sentence Window Query Engine,0.922222,0.70,0.613704,10.8,0.0
Direct Query Engine,0.890000,0.77,0.866833,10.8,0.0


In [53]:
tru.run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.
Dashboard already running at path:   Submit this IP Address: 35.229.251.83



<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

# **2. Auto-merging retrieval**

Auto-Merging Retrieval is a method that dynamically adjusts the granularity of information retrieval by organizing text into a hierarchical structure of chunks.

Hierarchy of Chunks:

**Parent Chunks** : Larger segments of text that encompass broader context.
**Smaller Chunks** : Subdivisions of the parent chunks that contain more granular information.
Merging Process:

Smaller chunks are linked to parent chunks.
When a certain threshold is exceeded (e.g., multiple smaller chunks linking to a parent chunk), these smaller chunks may be merged back into the larger parent chunk.

In [54]:
from llama_index.core.node_parser import HierarchicalNodeParser
from llama_index.core.node_parser import get_leaf_nodes
from llama_index.core.retrievers import AutoMergingRetriever
from llama_index.core.query_engine import RetrieverQueryEngine


def build_automerging_index(
    documents,
    llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    save_dir="merging_index",
    chunk_sizes=None,
):
    chunk_sizes = chunk_sizes or [2048, 512, 128]
    node_parser = HierarchicalNodeParser.from_defaults(chunk_sizes=chunk_sizes)
    nodes = node_parser.get_nodes_from_documents(documents)
    leaf_nodes = get_leaf_nodes(nodes)
    merging_context = ServiceContext.from_defaults(
        llm=llm,
        embed_model=embed_model,
    )
    storage_context = StorageContext.from_defaults()
    storage_context.docstore.add_documents(nodes)

    if not os.path.exists(save_dir):
        automerging_index = VectorStoreIndex(
            leaf_nodes, storage_context=storage_context, service_context=merging_context
        )
        automerging_index.storage_context.persist(persist_dir=save_dir)
    else:
        automerging_index = load_index_from_storage(
            StorageContext.from_defaults(persist_dir=save_dir),
            service_context=merging_context,
        )
    return automerging_index


def get_automerging_query_engine(
    automerging_index,
    similarity_top_k=12,
    rerank_top_n=2,
):
    base_retriever = automerging_index.as_retriever(similarity_top_k=similarity_top_k)
    retriever = AutoMergingRetriever(
        base_retriever, automerging_index.storage_context, verbose=True
    )
    rerank = SentenceTransformerRerank(
        top_n=rerank_top_n, model="BAAI/bge-reranker-base"
    )
    auto_merging_engine = RetrieverQueryEngine.from_args(
        retriever, node_postprocessors=[rerank]
    )
    return auto_merging_engine

In [55]:
automerging_index = build_automerging_index(
    documents,
    llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    save_dir="merging_index"
)

<ipython-input-54-350a78b13113>:18: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  merging_context = ServiceContext.from_defaults(


In [56]:
automerging_query_engine = get_automerging_query_engine(
    automerging_index,
)

In [57]:
auto_merging_response = automerging_query_engine.query(
    "How do I build a portfolio of AI projects?"
)
print(str(auto_merging_response))

> Merging 1 nodes into parent node.
> Parent node id: b8f1904d-eefd-4576-b11e-d09dcf2326eb.
> Parent node text: PAGE 21Building a Portfolio of 
Projects that Shows 
Skill Progression CHAPTER 6
PROJECTS

> Merging 1 nodes into parent node.
> Parent node id: 51d36528-0e76-4838-bdc9-f9f8d2c47919.
> Parent node text: PAGE 21Building a Portfolio of 
Projects that Shows 
Skill Progression CHAPTER 6
PROJECTS

Building a portfolio of AI projects involves showcasing a progression from simple to complex undertakings over time. It is important to be able to effectively communicate your thinking to demonstrate the value of your work and gain the trust of others. Identifying ideas that are worth working on is a crucial skill for an AI architect, and gaining experience by working on projects across various industries can help in building a diverse portfolio.


In [58]:
#tru.reset_database()

tru_recorder_automerging = get_prebuilt_trulens_recorder(automerging_query_engine,
                                                         app_id="Automerging Query Engine")

In [59]:
for question in eval_questions:
    with tru_recorder_automerging as recording:
        response = automerging_query_engine.query(question)
        print(question)
        print(response)

> Merging 2 nodes into parent node.
> Parent node id: 57f77b2a-ba16-4374-819a-03e8520a5070.
> Parent node text: PAGE 3Table of 
ContentsIntroduction: Coding AI is the New Literacy.
Chapter 1: Three Steps to Ca...

> Merging 1 nodes into parent node.
> Parent node id: da433608-75fb-4b6b-a6fd-69cc9885b4dc.
> Parent node text: PAGE 3Table of 
ContentsIntroduction: Coding AI is the New Literacy.
Chapter 1: Three Steps to Ca...

What are the keys to building a career in AI?
Learning foundational technical skills, working on projects, finding a job, and being part of a community are key steps to building a career in AI. Additionally, collaborating with others, influencing, and being influenced by others are critical aspects for success in AI.
How can teamwork contribute to success in AI?
Teamwork can contribute to success in AI by enabling individuals to work more effectively on large projects. Collaborating with others allows for a diversity of perspectives and skills to be brought together

In [60]:
tru.get_leaderboard(app_ids=[])

,Answer Relevance,Context Relevance,Groundedness,latency,total_cost
app_id,,,,,
Automerging Query Engine,0.911111,0.855556,0.811111,10.8,0.0
Sentence Window Query Engine,0.910000,0.730000,0.652333,10.8,0.0
Direct Query Engine,0.890000,0.770000,0.866833,10.8,0.0


# **Auto-Merging Query Engine:**

Strengths: Excellent in context relevance and answer relevance, making it a strong choice when the broader context of the information is crucial.

Weaknesses: Slightly lower groundedness compared to the Direct Query Engine, which may be important in scenarios where factual accuracy is paramount.

# **Sentence Window Query Engine:**

Strengths: High answer relevance.
Weaknesses: Lower context relevance and groundedness, indicating it might struggle to maintain the overall context and factual basis when retrieving smaller chunks of text.

# **Direct Query Engine:**

Strengths: Best at maintaining the grounding of responses, making it the most reliable for factually accurate information retrieval.
Weaknesses: Slightly lower in context relevance and answer relevance compared to the Auto-Merging Query Engine, but still very strong overall.


Overall, the Auto-Merging Query Engine is highly effective with proper tuning and eval scores differ and has the best


In [61]:
# launches on http://localhost:8501/
tru.run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.
Dashboard already running at path:   Submit this IP Address: 35.229.251.83



<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

In [62]:
tru.run_dashboard(port=8501, force=True)

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.
Dashboard closed.
Dashboard closed.
Go to this url and submit the ip given here. your url is: https://huge-turtles-watch.loca.lt

  Submit this IP Address: 35.229.251.83



<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>